In [1]:
#verification that each region yileds the equilibrium suggested by theory
%load_ext autoreload
%autoreload 2
from utils.imports import *
from utils.single_stage import *
from dsGameSolver.gameSolver import dsSolve

In [2]:
region=3
ca,cb,la,lb,paa_t,pba_t,pbb_t,pab_t = get_example(region)
dist  = 'uniform'
maxpx = ca+5
npts = 100

In [3]:
objaa_t, objba_t, objbb_t, objab_t = get_objs(paa_t,pba_t,pbb_t,pab_t,ca,cb,la,lb,dist)
paa_arr,pba_arr,objaa,objba,_ = get_payoffs_a(ca,cb,maxpx,npts,dist,la)
pbb_arr,pab_arr,objbb,objab,_ = get_payoffs_b(ca,cb,maxpx,npts,dist,lb)

In [4]:
def get_computed_equilibrium(payoffs,p1_arr,p2_arr):
    equilibrium = dsSolve(payoffs)
    eq_indices = np.argmax(equilibrium['strategies'][0],axis=1)
    temp = [p1_arr[eq_indices[0]],p2_arr[eq_indices[1]],equilibrium['stateValues'][0][0],equilibrium['stateValues'][0][1]]
    return (np.round(x,3) for x in temp)

In [5]:
paa_c,pba_c,objaa_c,objba_c = get_computed_equilibrium([np.array([objaa,objba])],paa_arr,pba_arr)
pbb_c,pab_c,objbb_c,objab_c = get_computed_equilibrium([np.array([objbb,objab])],pbb_arr,pab_arr)

print('A\'s strong sub-market (theoretical:paa,pba,objaa,objba,constraintA):',
      paa_t,pba_t,objaa_t,objba_t,constraints_a(paa_t,pba_t,ca,cb,la))
print('A\'s strong sub-market (computed:paa,pba,objaa,objba,constraintA):',
      paa_c,pba_c,objaa_c,objba_c,constraints_a(paa_c,pba_c,ca,cb,la))
print('B\'s strong sub-market (theoretical:pbb,pab,objbb,objab,constraintB):',
      pbb_t,pab_t,objbb_t,objab_t,constraints_b(pbb_t,pab_t,cb,ca,lb))
print('B\'s strong sub-market (computed:pbb,pab,objbb,objab,constraintB):',
      pbb_c,pab_c,objbb_c,objab_c,constraints_b(pbb_c,pab_c,cb,ca,lb))

A's strong sub-market (theoretical:paa,pba,objaa,objba,constraintA): 1.98 0.99 0.495 0.49 True
A's strong sub-market (computed:paa,pba,objaa,objba,constraintA): 2.01 1.03 0.515 0.505 True
B's strong sub-market (theoretical:pbb,pab,objbb,objab,constraintB): 1.65 1.32 1.378 0.053 True
B's strong sub-market (computed:pbb,pab,objbb,objab,constraintB): 1.697 1.354 1.406 0.061 True
